In [2]:
#Import the libraries

import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import datetime
from datetime import timedelta, date
import re

In [4]:
#Get the webpage and parse it as html

page = requests.get("https://forecast.weather.gov/MapClick.php?lat=37.777120000000025&lon=-122.41963999999996#.YFs1e6_7QdW")
soup = BeautifulSoup(page.content, 'html.parser')
html = list(soup.children)[2]
body = list(html.children)[3]        # Required data is nested here

In [7]:
#Retrieve the day names nested in body 

day_names = body.find_all('div', class_ = 'col-sm-2 forecast-label') #day names are in this tag
list_of_days = []

for each_day in day_names:
    e = each_day.text                #'.text' returns the text inside the Tag
    list_of_days.append(e)
print(list_of_days)

#dict['Day'] = list_of_days
#dict.to_csv('')                     #tansform to csv so that others can import and work on the file

['Overnight', 'Thursday', 'Thursday Night', 'Friday', 'Friday Night', 'Saturday', 'Saturday Night', 'Sunday', 'Sunday Night', 'Monday', 'Monday Night', 'Tuesday', 'Tuesday Night', 'Wednesday']


In [30]:
#Retrieve the weather summary nested inside the body and find temperatures

day_summary = body.find_all('div', class_ = 'col-sm-10 forecast-text') #day summary is in this tag

summary_list = []
for summary in day_summary:
    e = summary.text              
    summary_list.append(e)
    
df = pd.DataFrame(summary_list)     

summary_string = ' '.join(summary_list)              #convert list to string to retrieve other information
temp = re.findall("[0-9]\d[.]", summary_string)      #regex searches the temp. Eg. '40.' pattern
temp = ' '.join(temp).replace('.','').split()        #remove the period at the end of each number


['48', '62', '46', '68', '47', '70', '48', '68', '48', '64', '47', '69', '49', '72']


In [48]:
#Convert Fahrenheit to celcius 
temp_celcius = []
def celcius(temp):
    for each_temp in temp:
        each_temp = int(each_temp)*(5/9)         #converting to int type as it gives an error for float
        new_temp = round(each_temp)
        temp_celcius.append(f'{new_temp} °C')
    return temp_celcius
#celcius(temp)

In [49]:
#create a dynamic date format

todays_date = date.today()
final_date = todays_date + timedelta(days=6)
delta = todays_date - final_date                     # make a delta for range
date_range = pd.date_range(start=todays_date, end=final_date) 
date_range = date_range.repeat(2)                    #Each day has morning and evening
#print(date_range)